Task3 - reassesed the requirements, now using dataframes more

In [18]:
import pandas as pd
import os, glob, re
from glob import glob
from bs4 import BeautifulSoup
import logging, importlib, sys, tqdm
import spacy
from spacy.lang.en import English
from spacy.lang.en.stop_words import STOP_WORDS
from sklearn.feature_extraction.text import CountVectorizer
from _logging import set_logging
from save_file import save_file

set_logging(logging)

In [19]:
def read_html_file(fname):
    with open (fname, 'r', encoding="utf8") as f:
        doc = BeautifulSoup(f.read(), 'html.parser')
        return doc

In [20]:
def tokenise_words(text, nlp):
    word_tokens = nlp(text)
    # converts the words in word_tokens to lower case and then checks whether
    # they are present in stop_words or not
    tokens = [w for w in word_tokens if not w.is_stop and not w.is_punct and not w.is_space]

    return tokens

In [21]:
def get_body(tag_body):
    body = " ".join(tag_body.strings)
    return body

In [22]:
def get_bullet_points(tag_body):
    tag_bullet_point = "li"
    bullet_points = ""
    for item in tag_body.find_all(tag_bullet_point):  
        bullet_points += item.get_text(separator=" ")
    return bullet_points

Read the resume

In [23]:
#nlp = English()
nlp = spacy.load('en_core_web_md')
file_path = os.path.join("Files\\UserInput", 'resume.txt')
f = open(file_path, "r")
resume_text = f.read()
nlp_resume_text = nlp(resume_text)

Get the cosine similarity score between the resume and the job description

In [24]:
def get_cosine_similarity_score(body, nlp_resume_text, nlp):
    nlp_job = nlp(body)
    score = nlp_resume_text.similarity(nlp_job)
    return score

Load all the job descriptions from the HTML files into a dataframe

In [25]:
files = glob(os.path.join("Files\\Input", '*.html'))
col_names =  ['Id', 'Title', 'Body', 'Bullet_Points', 'Score']
df_jobs = pd.DataFrame(columns = col_names)

for i, fname in enumerate(files):
    message = f"{i+1}. file : {fname}"
    logging.info(message)
    doc = read_html_file(fname)
    title = doc.title.string
    body = get_body(doc.body)
    bullet_points = get_bullet_points(doc.body)
    score = get_cosine_similarity_score(body, nlp_resume_text, nlp)
    df_jobs.loc[len(df_jobs)] = [i, title, body, bullet_points, score]

q = 0